In [1]:
from ktrain import text
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
ts = text.TransformerSummarizer(device="cuda")

In [2]:
df = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/test_small.csv")

In [6]:
def summarize(text):
    return ts.summarize(text)

In [10]:
sample = df.sample(100, random_state=42)

In [11]:
summaries = sample['text'].progress_apply(summarize)

In [12]:
summaries

119737    Marie is very friendly, and is detail oriented...
72272     Chocolate chip cake with white cream cheese fr...
158154    The food was great and Josh, our server was aw...
65426     The Original Philly Sandwich from Philly's Fam...
30074     The pizza was amazing. The service was quick a...
                                ...                        
182091    The Fertility Center of Las Vegas has the high...
31929     Great food especially the goat curry and mango...
78446     Jedd's is one of my FAVORITE places to crunch ...
145114    We used the $49 voucher for 2 - each includes ...
96761     The food was amazing and the service was quick...
Name: text, Length: 100, dtype: object

In [13]:
sample['summaries'] = summaries

In [17]:
sample['str_len'] = sample['text'].str.len()

In [19]:
sample.sort_values('str_len', ascending=False).head()

,text,stars,pos_neg_neu,pos_neg_3_is_pos,pos_neg_3_is_neg,summaries,str_len
182091,My wife and I (we are a lesbian couple) had al...,5.0,POSITIVE,POSITIVE,POSITIVE,The Fertility Center of Las Vegas has the high...,2994
41259,I'm from California down for Mother's Day week...,1.0,NEGATIVE,NEGATIVE,NEGATIVE,"""I waited an hour and a half and she wants me ...",2703
137859,Jumpin' in the Hood\n\nI had never heard of th...,1.0,NEGATIVE,NEGATIVE,NEGATIVE,The trampolines are made of a more slippery ma...,1741
164774,We had to make the very unfortunate decision t...,5.0,POSITIVE,POSITIVE,POSITIVE,"""We had to make the very unfortunate decision ...",1484
181498,A friend of mine had been raving with love for...,4.0,POSITIVE,POSITIVE,POSITIVE,A friend of mine had been raving with love for...,1428


In [29]:
print(sample.loc[164774]['text'])

We had to make the very unfortunate decision to put my cat to sleep this morning. I've had him for 14 years so this has been a very difficult decision. I called at 8:30am and they got us in at 9:30am. My husband went in and paid and signed the form and came out to get me and my cat. My husband said it was so nice inside and didn't feel like a vets office at all.  We went into a side room away from the lobby. What I noticed most was that my cat wasn't laying on a cold metal table.  Instead there was a nice soft blanket waiting for him. They took him and did a paw print and got the catheter started.  We were given as much time as needed.  Dr. Bestic was wonderful.  I've euthanized pets in the past but this was the absolute best experience.  He went so quickly and peacefully. They gave us as much time as we needed afterwards.  I chose to have him privately cremated with Paws Awhile which they take care of for me.  He will be returned to me in about a week in a nice wood box with his name 

In [30]:
print(sample.loc[164774]['summaries'])

"We had to make the very unfortunate decision to put my cat to sleep this morning. I've had him for 14 years so this has been a very difficult decision" "Dr. Bestic was wonderful.  I've euthanized pets in the past but this was the absolute best experience. He went so quickly and peacefully"


In [24]:
from transformers import pipeline

In [25]:
sentiment = pipeline("sentiment-analysis")

In [31]:
sentiment(sample.loc[164774]['summaries'])

[{'label': 'POSITIVE', 'score': 0.9985255002975464}]

In [36]:
doc = """I have passed by this place a couple of times and wondered what kind of place this was. I did go to it in April, but I was there for the special burger because it was after Ebertfest. It was good, but I wanted to come back for the real deal dinner of chicken and such. 

I would argue that this is one of the best restaurants in all of Champaign. 

My dad and I were down in Champaign for the Illini Football game. I remembered wanting to try this place and we knew they had some pretty generous hours so we would be able to catch dinner. 

My dad went with a couple of craft beers, but I tried their palaoma. They are not shy about the booze so the $12 price tag was worth it! 

We started with the Pimento Cheese appetizer which was a jar of pepery cheese spread served with crackers, pickles, and celery. I didn't think we had enough crackers at the beginning, but there was actually a perfect amount. The cheese also had a kick, but was not overly spicy which my dad liked. 

For dinner, my dad got the pulled chicken sandwich. He let me try some of it and it was extremely flavorful and juicy. He also liked that it had a taste, but wasn't spicy. I went the other end and got the Nashville Hot quarter chicken. It was juicy as well and nice and spicy. It was very spicy, but not super spicy. They also didn't drown the bird so the sauce was more like a rub on the skin. This kept the chicken crispy! 

If you are looking for a great late night bite, this is a great option! They have a great dinner, brunch, and late night menu. Check it out!"""

print(ts.summarize(doc))

I would argue that this is one of the best restaurants in all of Champaign. The Pimento Cheese appetizer was a jar of pepery cheese spread served with crackers, pickles, and celery. The Nashville Hot quarter chicken was juicy as well and nice and spicy.
